In [ ]:
## Regression Script

# Script Name: RegressionScript.ipynb
# Purpose: Provide an example of a regression model
# Author: K21014303
# Date: Last edited 20/11/24


In [4]:
# Preprocessing 
from sklearn.preprocessing import (
    StandardScaler, 
    MinMaxScaler, 
    RobustScaler, 
    LabelEncoder, 
    PowerTransformer
)
from sklearn.utils import class_weight
from sklearn.experimental import enable_halving_search_cv

# Model Selection and evaluation
from sklearn.model_selection import (
    train_test_split, 
    StratifiedKFold, 
    StratifiedShuffleSplit, 
    KFold, 
    cross_val_score, 
    cross_val_predict, 
    GridSearchCV,
    HalvingGridSearchCV
)
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Modelling 
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from tensorflow.keras.utils import to_categorical

# Pipeline 
from sklearn.pipeline import Pipeline


In [1]:
## Loading data 

# Loading the .CSV file
data = pd.read_csv('data.csv') # Cannot be provided on github as has confidential data 

# Checking the data shape
print(data.shape) 

# Inspecting the first few rows of the data
print(data.head())

# The data is structured with participant ID, Age, Diagnosis, MRI Field Strength, and remaining 292 columns are MRI-derived features

NameError: name 'pd' is not defined

In [2]:
## Preparing data 

# Dropping non-numeric variables
number_data = data.drop(columns=['ID', 'Age', 'Diagnosis', 'FieldStrength']) 
feature_names = number_data.columns.tolist()  # Saving the variable names for later use 

# Encoding the diagnostic labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(data['Diagnosis'])


NameError: name 'data' is not defined

In [3]:
## Exploratory data analysis
# Compute the correlation matrix
corr_matrix = number_data.corr()

# Plot the correlation matrix using a heatmap
plt.figure(figsize=(20, 15))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

# Correlation matrix shows most of our MRI features are correlated except the asymmetry features. This suggests we should be aware of 
# multicolinearity with our highly correlated features. 

NameError: name 'number_data' is not defined

In [ ]:
## Splitting the dataset into training (80%) and testing (20%) sets to evaluate the model performance on unseen data 
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Generating the training/testing splits
for train_index, test_index in sss.split(number_data, y_encoded):
    X_train, X_test = number_data.iloc[train_index], number_data.iloc[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]

In [ ]:
## Logistic Regression

# Proposed pipeline: 
# Scaling -> Dimensionality Reduction (PCA) -> Feature Selection (RFE) -> Regularisation (L2) -> Cross Validation (k = 5) -> Model Evaluation 

# (Logistic regression is not optimal for the classification of MRI features due to nonlinearly separable classes, but is being used here to show
# an example of code structure)

In [ ]:
## Proposed pipeline

# Dimensionality Reduction using PCA with visual analysis of optimal component number first 
scaler = StandardScaler()                      # Scaling the data with StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
pca = PCA()
X_train_pca = pca.fit_transform(X_train_scaled)      # Fitting PCA
X_test_pca = pca.transform(X_test_scaled)

# Visualising the explained variance
explained_var = pca.explained_variance_ratio_

# Plotting explained variance and cumulative explained variance
# Explained variance with red
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(explained_var, 'r')
ax.set_title('Explained Variance by Principal Components', fontsize=20)
ax.set_xlabel('Number of Principal Components', fontsize=14)
ax.set_ylabel("Explained Variance", color='r', fontsize=14)

# Plot cumulative explained variance
ax2 = ax.twinx()
ax2.plot(np.cumsum(explained_var), 'b')
ax2.set_ylabel('Cumulative Explained Variance', color='b', fontsize=14)

plt.show()

# Visual inspection of plotting explained variance against cumulative explained variance shows that the optimal number of components 
# should be under 5. However, with MRI data using more components can retain critical features and improve classifier performance, at the
# cost of potential overfitting or redundancy. So we will use a gridsearch to optimise the number of components for accuracy. 

In [ ]:
## Running Pipeline with HalvingGridSearchCV (performed on spyder)

# The HalvingGridSearchCV will provide the optimal parameters for increasing model performance, whilst iteratively narrowing down 
# the hyperparameter space. 
# In the best circumstances, we would use a normal GridSearchCV and test all the possible parameters, but we cannot do this due to 
# resource management and efficiency.

# Defining the Logistic Regression model with ridge regularisation 
LRmodel = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42, penalty='l2') 
# Class weight is balanced due to unbalanced weighting of diagnosis in data
# L2 penalty applied to control overfitting, improve generalization, stabilise coefficient estimates and manage multicollinearity
# L1 was not applied as it is less stable with multicollinearity and MRI features are often highly correlated 

# Defining the parameter grid for GridSearchCV including scaler options, PCA component numbers and RFE feature selection
LRparam_grid = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler()], # Options for scalers 
    'pca__n_components': range(1, 292),                           # Range for PCA component selection to reflect the number of MRI features
    'rfe__n_features_to_select': range(1, 292)                    # Range for Feature selection chosen to reflect the number of MRI features
}                                                                  

# Create a pipeline that includes PCA and RFE
LRpipeline = Pipeline([
    ('scaler', 'passthrough'),  # Passthrough acts as a placeholder for scaler evaluation 
    ('pca', PCA()),             # PCA dimensionality reduction 
    ('rfe', RFE(LRmodel)),      # RFE feature selection 
    ('classifier', LRmodel)     # Logistic regression classification model 
])

# Setting up HalvingGridSearchCV with cross-validation 
LRgrid_search = HalvingGridSearchCV(
    LRpipeline,
    param_grid=LRparam_grid,
    scoring='accuracy',         # Provides the accuracy of each tested model
    cv=5,                       # Number of cross-validation folds included 
    n_jobs=5,                   # Perform multiple jobs at once 
    verbose=10                  # Provides progress updates
)

# Fitting the GridSearchCV to our training data
LRgrid_search.fit(X_train, y_train)

# Best parameters and cross-validated accuracy taken from the GridSearchCV
LRbest_params = LRgrid_search.best_params_
LRbest_accuracy = LRgrid_search.best_score_

print(f'Best parameters: {LRbest_params}')
print(f'Best cross-validated accuracy: {LRbest_accuracy:}') 

# Final model training with the best parameters 
LRbest_model = LRgrid_search.best_estimator_

# Making predictions on the test set
y_pred = LRbest_model.predict(X_test)

# Confusion Matrix to present the accuracy 
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='hot', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()
# Diagonal cells should all be closest to white/yellow for the best classification of true labels. 

# Classification Report for easy interpretation of model evaluation alongside grid search 
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Output results
# Best parameters: {'pca__n_components': 47, 'rfe__n_features_to_select': 153, 'scaler': RobustScaler()}
# Best cross-validated accuracy: 0.5409554482018251
#               precision    recall  f1-score   support

#           AD       0.49      0.71      0.58        52
#           CN       0.46      0.63      0.53        79
#          MCI       0.57      0.34      0.43       137

#     accuracy                           0.50       268
#    macro avg       0.50      0.56      0.51       268
# weighted avg       0.52      0.50      0.49       268